In [1]:
import xarray as xr
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import os
import glob
import shutil
import matplotlib.gridspec as gridspec
from scipy import interpolate
from scipy.interpolate import interp2d
from mpl_toolkits.basemap import Basemap

Initial data: gridded SSHA data from CMEMS, available at https://data.marine.copernicus.eu/product/SEALEVEL_GLO_PHY_L4_MY_008_047/description .  
“Generated using E.U. Copernicus Marine Service Information; <https://doi.org/10.48670/moi-00148>”

For visualisation purposes, I took out daily SSHA files, relevant for April 2014.
The script performs following procedures:
1. Renames original data according to respective days, sorts them and stores in a user-defined directory 'output_dir'
2. Outliers are removed, using the percentile method - the outlliers are prescribed the values of respective thresholds
3. NaNs are saved as a mask and converted to 0 in the original array
4. The array is subjected to 2D cubic interpolation (smoothes outliers set to threshold values, deals with NaNs that we converted to zeroes)
6. Then the interpolation is assessed on a finer grid (0.25x0.25deg. original resolution vs 0.1x0.1deg. new resolution)
7. Raw and interpolated data are plotted on basemaps, Drake passage region is chosen as an example
8. Maps are saved in .jpg format in 'map_dir'
9. Maps are combined in .gif and saved in 'output_dir'

In [2]:
data_dir = 'C:/Users/vavolk001/Desktop/Thesis/data/CMEMS/SSH_gridded/2014/04' # the directory where original .nc files with SSHA data are stored. Unzip the data, place it wherever is suitable, change the directory up to the last folder
map_dir = 'C:/Users/vavolk001/Desktop/climAPC/maps/' # the directory where plotted SSHA maps are stored
output_dir = 'C:/Users/vavolk001/Desktop/climAPC/' # the directory where renamed sorted .nc data and final .gif is stored

In [3]:
title_fontsize = 66
label_fontsize = 52
colorbar_fontsize = 50

In [4]:
# Get a list of netCDF files in the data directory
file_list = [f for f in os.listdir(data_dir) if f.endswith('.nc')]
# Sort the file list alphabetically
file_list.sort()

# Loop through each file and rename it
for idx, file_name in enumerate(file_list):
    # Extract the day number from the index
    day_number = str(idx + 1).zfill(2)  # Convert index to 2-digit string
    # Construct the new file name
    new_file_name = day_number + '_04_2014.nc'
    # Create the full file paths
    old_file_path = os.path.join(data_dir, file_name)
    new_file_path = os.path.join(output_dir, new_file_name)
    # Rename the file
    shutil.copy(old_file_path, new_file_path)

In [5]:
# Get a list of netCDF files in the output directory
file_list = [f for f in os.listdir(output_dir) if f.endswith('.nc')]
# Sort the file list alphabetically
file_list.sort()

for file_name in file_list:
    fig,ax = plt.subplots(2,sharex=True,figsize=(102, 112))
    gs = gridspec.GridSpec(2, 2, height_ratios=[3, 1])
    fig.suptitle('Raw vs Processed vs Processed Interpolated SSHA')
    contour_levels = np.linspace(-0.5,0.5, 20)
    cmap = 'seismic'
    parallels = np.arange(80.,-81,-20.)
    #labels = [left,right,top,bottom]
    meridians = np.arange(10.,351.,20.)
    
    # 0. Loading data
    file_path = os.path.join(output_dir, file_name)
    data = xr.open_dataset(file_path, engine='netcdf4')
    dynamic_topography = data['sla'].values
    dyn_topo_raw = np.squeeze(dynamic_topography, axis=0)
    latitude_raw = data['latitude'].values
    longitude_raw = data['longitude'].values
 
    # 1. Removing outliers
    # Define the lower and upper percentile thresholds
    low_perc = 5  # lower percentile threshold (e.g., 5%)
    up_perc = 95  # upper percentile threshold (e.g., 95%)
    # Calculate the percentiles along the latitude axis (axis=0)
    low_thresh = np.max(np.nanpercentile(dyn_topo_raw, low_perc))
    up_thresh = np.min(np.nanpercentile(dyn_topo_raw, up_perc))
    # Remove outliers based on the percentile thresholds
    dyn_topo_filt = np.where((dyn_topo_raw < low_thresh), low_thresh, dyn_topo_raw)
    dyn_topo_filt = np.where((dyn_topo_raw > up_thresh), up_thresh, dyn_topo_raw)
    
    # 2. Removing missing values
    lon, lat = np.meshgrid(longitude_raw, latitude_raw)
    mask = np.isnan(dyn_topo_filt) #masking coarse grid
    filled_dyn_topo = np.where(mask, 0, dyn_topo_filt) #fill that mask with zeroes
    dyn_topo = interp2d(longitude_raw, latitude_raw, filled_dyn_topo, kind='cubic') #filled interpolation, coarse grid
    dt_interp = dyn_topo(longitude_raw, latitude_raw) #we get that interpolation, coarse grid
    
    # 3. Create a new, finer 0.1x0.1 grid
    targ_res = 0.1 # target resolution
    targ_lon = np.arange(longitude_raw[0], longitude_raw[-1] , targ_res)
    targ_lat = np.arange(latitude_raw[0], latitude_raw[-1] , targ_res)
    
    # 4. Interpolate on a finer grid
    fine_interp = dyn_topo(targ_lon, targ_lat)
    
    # 5. Initiate basemap and corresponding coordinates
    m=Basemap(projection='cyl',llcrnrlon=-110, llcrnrlat=-80, urcrnrlon=-10, urcrnrlat=-20)
    t_lat, t_lon = m(targ_lat, targ_lon)
    latitude,longitude = m(lat,lon)
    t_lon,t_lat = np.meshgrid(t_lon,t_lat)
    
    # 6. Plotting raw vs processed and interpolated data 
    # Raw data
    fig.subplots_adjust(right=0.9)
    m0 = Basemap(projection='cyl',llcrnrlon=-110, llcrnrlat=-80, urcrnrlon=-10, urcrnrlat=-20)
    m0.ax = ax[0]
    m0.drawparallels(parallels,labels=[False,True,False,False], color='0.8')
    m0.drawmeridians(meridians,labels=[True,False,False,True], color='0.8')
    a=m0.contourf(longitude, latitude, dyn_topo_raw, cmap='seismic', levels=contour_levels, extend='both')
    m0.fillcontinents(color='white')
    m0.drawcoastlines()

    # Processed and interpolated data
    m1 = Basemap(projection='cyl',llcrnrlon=-110, llcrnrlat=-80, urcrnrlon=-10, urcrnrlat=-20)
    m1.ax = ax[1]
    m1.drawparallels(parallels,labels=[False,True,False,False], color='0.8')
    m1.drawmeridians(meridians,labels=[True,False,False,True], color='0.8')
    b=m1.contourf(t_lon, t_lat, fine_interp, cmap='seismic', levels=contour_levels, extend='both')
    m1.drawcoastlines()
    m1.fillcontinents(color='white')
      
    cbar=fig.colorbar(a, ax=ax.ravel().tolist(),aspect=40)
    cbar.set_label('Sea surface height anomaly, m', fontsize=label_fontsize)
    
    ax[1].set_xlabel('Longitude', labelpad=20,fontsize=label_fontsize)
    ax[0].set_ylabel('Latitude', labelpad=20,fontsize=label_fontsize)
    ax[1].set_ylabel('Latitude', labelpad=20,fontsize=label_fontsize)
    
    # 7. Saving maps
    fig.savefig(os.path.join(map_dir, f'{file_name}.jpg'))
    plt.close(fig)

C:\Users\vavolk001\Anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [13]:
# 8. Saving maps into a gif
import os
from PIL import Image
output_dir = map_dir+'merged_maps.gif'
def merge_maps_into_gif(map_dir, output_dir):
    # Get a list of all .jpg files in the map directory
    file_list = [f for f in os.listdir(map_dir) if f.endswith('.jpg')]
    # Sort the file list alphabetically
    file_list.sort()

    images = []
    for file_name in file_list:
        file_path = os.path.join(map_dir, file_name)
        image = Image.open(file_path)
        images.append(image)

    # Save the images list as a GIF file
    images[0].save(output_dir, format='GIF', save_all=True, append_images=images[1:], duration=0.1, loop=0)

# Call the merge_maps_into_gif function
merge_maps_into_gif(map_dir, output_dir)